Collect & Import all the libraries once before coding

In [2]:
import seaborn as sns
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import cv2
import random 
from sklearn.model_selection import train_test_split
import shutil
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from keras import models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay


## EDA och filhantering 

#### Läs in 10 bilder slumpmässigt, plotta dem och extrahera deras labels och skriv ut i titeln.


In [3]:
def plot_n_random_img(n, path):

    # Seclect sample images
    samples = random.sample(os.listdir(path), n)

    # loop the path for images
    path_images = [f"{path}/{file_name}" for file_name in samples]

    # read the images to array
    images_read = [plt.imread(file) for file in path_images]

    # Display the images with labels
    fig, axes = plt.subplots(2, int(n/2), figsize=(15, 6))
    for i, ax in enumerate(axes.flatten()):
        ax.imshow(images_read[i])
        ax.set_title(f"{samples[i]}")
        ax.axis("off")
    
    plt.suptitle(f"{n} random images from cats and dogs")


path = "../input/dogs-vs-cats/train/train"
plot_n_random_img(10, path)

#### Skapa folderstrukturen nedan med Python:

Create the dir (train/val/test) for small data:

In [4]:
experiment_small_data = "experiment_small_data"
os.mkdir(experiment_small_data)

# For small train

small_train_dir = os.path.join(experiment_small_data, "train")
os.mkdir(small_train_dir)

# For small test

small_test_dir = os.path.join(experiment_small_data, "test")
os.mkdir(small_test_dir)

# For small val 

small_val_dir = os.path.join(experiment_small_data, "val")
os.mkdir(small_val_dir)

Create the dir (train/test) for tiny data:

In [5]:
experiment_tiny_data = 'experiment_tiny_data'
os.mkdir(experiment_tiny_data)

# For tiny train

tiny_train_dir = os.path.join(experiment_tiny_data, "train")
os.mkdir(tiny_train_dir)

# For tiny test

tiny_test_dir = os.path.join(experiment_tiny_data, "test")
os.mkdir(tiny_test_dir)

# For tiny val 

tiny_val_dir = os.path.join(experiment_tiny_data, "val")
os.mkdir(tiny_val_dir)

#### Nu ska göra train|val|test split med följande splits:

experiment_small:
- train - 1600 (800 dogs, 800 cats)
- val - 400 (200 dogs, 200 cats)
- test - 500 (250 dogs, 250 cats)

experiment_tiny (BONUS):
- train - 160 (80 dogs, 80 cats)
- val - 40 (20 dogs, 20 cats)
- test - 50 (25 dogs, 25 cats)



In [6]:
# Check how many train pics
train_img = os.listdir("../input/dogs-vs-cats/train/train")
len(train_img)

Create a total name's pool for dogs and cats

In [7]:
train_cats = ["cat.{}.jpg".format(i) for i in random.sample(range(0, 12500), 12500)]
train_dogs = ["dog.{}.jpg".format(i) for i in random.sample(range(0, 12500), 12500)]

Original folder for img

In [8]:
original_dir = "../input/dogs-vs-cats/train/train"

Define a function for split

In [9]:
def train_val_test_split(n_train, n_val, n_test, path_train, path_val, path_test):

    # here randomly select n images from 1st-8000st, 8000st-10000st, 10000st-12500st for train, val and test. 
    # The purpose of this method is to select samples segmently and to avoid data leaking

    train_cat = random.sample(train_cats[0:8000], int(n_train/2))
    train_dog = random.sample(train_dogs[0:8000], int(n_train/2))

    val_cat = random.sample(train_cats[8000:10000], int(n_val/2))
    val_dog = random.sample(train_dogs[8000:10000], int(n_val/2))

    test_cat = random.sample(train_cats[10000:12500], int(n_test/2))
    test_dog = random.sample(train_dogs[10000:12500], int(n_test/2))

    for cats, dogs in zip(train_cat, train_dog):
        src = os.path.join(original_dir, cats)
        dst = os.path.join(path_train, cats)
        shutil.copyfile(src, dst)
        src = os.path.join(original_dir, dogs)
        dst = os.path.join(path_train, dogs)
        shutil.copyfile(src, dst)
            
    for cats, dogs in zip(val_cat, val_dog):
        src = os.path.join(original_dir, cats)
        dst = os.path.join(path_val, cats)
        shutil.copyfile(src, dst)
        src = os.path.join(original_dir, dogs)
        dst = os.path.join(path_val, dogs)
        shutil.copyfile(src, dst)

    for cats, dogs in zip(test_cat, test_dog):
        src = os.path.join(original_dir, cats)
        dst = os.path.join(path_test, cats)
        shutil.copyfile(src, dst)
        src = os.path.join(original_dir, dogs)
        dst = os.path.join(path_test, dogs)
        shutil.copyfile(src, dst)


For small data:

In [10]:
small_train_dir ="./experiment_small_data/train"
small_val_dir ="./experiment_small_data/val"
small_test_dir ="./experiment_small_data/test"

train_val_test_split(1600, 400 ,500, small_train_dir, small_val_dir, small_test_dir)


In [11]:
train_small = os.listdir("./experiment_small_data/train")
val_small = os.listdir("./experiment_small_data/val")
test_small = os.listdir("./experiment_small_data/test")


len(train_small), len(val_small), len(test_small)

For tiny data:

In [12]:
tiny_train_dir ="./experiment_tiny_data/train"
tiny_val_dir ="./experiment_tiny_data/val"
tiny_test_dir ="./experiment_tiny_data/test"

train_val_test_split(160, 40 ,50, tiny_train_dir, tiny_val_dir, tiny_test_dir)

In [13]:
train_tiny = os.listdir("./experiment_tiny_data/train")
val_tiny = os.listdir("./experiment_tiny_data/val")
test_tiny = os.listdir("./experiment_tiny_data/test")


len(train_tiny), len(val_tiny), len(test_tiny)

#### Läs in dataseten från experiment_small, experiment_tiny och plocka ut labelsvektorer, som ska vara one-hot encoded med 0 och 1.

- plotta några bilder med deras respektive labels och kontrollera att det är korrekt.
- skapa lämplig plot för att kontrollera att dataseten är balanserade
- skapa lämplig plot för att kontrollera att dataseten är slumpade (dvs inte ex [0, 0, ... 0, 1, 1, ..., 1]).

Firstly create one-hot labels and randomize the data with function to avoid 0,0,0...1,1,1...

In [14]:
def randomize_and_one_hot(path):

    list = os.listdir(path)
    
    # shuffle the file names, mix the cat and dog images
    random.shuffle(list)

    images = [f"{path}/{file_name}" for file_name in list]

    y = []
    X = []
    
    for name in images:
      if "cat" in name:
        y.append(0)
      elif "dog" in name:
        y.append(1)
        
      X.append(cv2.imread(name))
  
    return X, y


Apply the function to small and tiny files

In [15]:
small_train_dir ="./experiment_small_data/train"
small_val_dir ="./experiment_small_data/val"
small_test_dir ="./experiment_small_data/test"

X_train_small, y_train_small = randomize_and_one_hot(small_train_dir)
X_val_small, y_val_small = randomize_and_one_hot(small_val_dir)
X_test_small, y_test_small = randomize_and_one_hot(small_test_dir)

tiny_train_dir ="./experiment_tiny_data/train"
tiny_val_dir ="./experiment_tiny_data/val"
tiny_test_dir ="./experiment_tiny_data/test"

X_train_tiny, y_train_tiny = randomize_and_one_hot(tiny_train_dir)
X_val_tiny, y_val_tiny = randomize_and_one_hot(tiny_val_dir)
X_test_tiny, y_test_tiny = randomize_and_one_hot(tiny_test_dir)

Plot the images (check)

In [16]:
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
for i, ax in enumerate(axes.flatten()):
    ax.imshow(X_train_tiny[i])
    ax.set_title(f"{y_train_tiny[i]}")
    ax.axis("off")
plt.suptitle("10 cats and dogs images with 0/1 labels")

Plot the balance of the datas

In [17]:
datas = [y_train_small, y_val_small, y_test_small, y_train_tiny, y_val_tiny, y_test_tiny]
titles = ["Balance small train", "Balance small val", 
          "Balance small test", "Balance tiny train", 
         "Balance tiny val","Balance tiny test",]

fig, axes = plt.subplots(2, 3, figsize=(15, 8))

for ax, data, title in zip(axes.flatten(), datas, titles):
    sns.countplot(x=data, ax=ax)
    ax.set(title = title)
    ax.axis("on")
plt.suptitle("Number of Cats & dogs")


From above we can see the same number of dog and cat in different data groups

In [18]:
fig, axes = plt.subplots(2, 3, figsize=(15, 8))

for ax, data, title in zip(axes.flatten(), datas, titles):
    sns.scatterplot(data=data,linewidth=.2, ax=ax)
    ax.set(title = title)
    ax.axis("on")
plt.suptitle("Images distribution")

Here above the figure shows the dog and cat images are randomly distributed

# Bildbehandling

#### a) Skapa en plot för att visualisera bildstorlekarna i träningsdatan. Använd seaborns jointplot.

In [19]:
X_train_small[0].shape
# This is a image with 456*499(size) pixers

For small dataset

In [20]:
width_small = [X_train_small[i].shape[0] for i in range(len(X_train_small))]
length_small = [X_train_small[i].shape[1] for i in range(len(X_train_small))]

fig = sns.jointplot(x = width_small, y = length_small, xlim = 550, ylim = 550)
fig.set_axis_labels("Width", "Length")
plt.suptitle("JointPlot for small train data")

For tiny dataset

In [21]:
width_tiny = [X_train_tiny[i].shape[0] for i in range(len(X_train_tiny))]
length_tiny = [X_train_tiny[i].shape[1] for i in range(len(X_train_tiny))]

fig = sns.jointplot(x = width_tiny, y = length_tiny, xlim = 550, ylim = 550)
fig.set_axis_labels("Width", "Length")
plt.suptitle("JointPlot for tiny train data")

#### b) Välj en lämplig bildstorlek att ändra samtliga bilder till. Gör en analys och fundera på om du behöver slänga bilder. Hur kommer du fram till ditt val?

The images will have to be reshaped to modeling so that all images have the same shape. This is often a small square image. The image size defaults to keras is (256, 256) and we can use this setting in our case. Because cats/dogs images has a maximal size in 500 (width/length) and 256*256 can retain most of the information. (Maybe change to a smaller size if it running too slow)

Through the figure analysis of c), we can see that the image size in small/tiny are basically between 100 and 500. This means that the images basically retains the information we need. However, when we browse the images in folder with eyes, can also found very few images that are neither cats nor dogs (t.ex cat.5418). This may affect the accuracy. But due to the small number and the high workload of manual culling, I decided not to act.

#### c) Gör resize sådant att samtliga bilder är samma storlek och spara dem i numpy arrays med följande struktur:
<span style="color:grey">(samples, row, cols, color_channels)</span>.  


In [22]:
def resize(dataset):

	images_resized = []
	
	for data in dataset:
		
		resized = cv2.resize(data, (256, 256))
		images_resized.append(resized)
		
	images_resized = np.asarray(images_resized)
	
	return images_resized


In [23]:
X_train_small_resized = resize(X_train_small)
X_val_small_resized = resize(X_val_small)
X_test_small_resized = resize(X_test_small)

X_train_tiny_resized = resize(X_train_tiny)
X_val_tiny_resized = resize(X_val_tiny)
X_test_tiny_resized = resize(X_test_tiny)

X_train_small_resized.shape, X_train_tiny_resized.shape

#### Visualisera därefter ett par styckena bilder.

In [24]:
fig, axes = plt.subplots(1, 4, figsize=(15, 6))
for i, ax in enumerate(axes.flatten()):
    ax.imshow(X_train_small_resized[i])
    ax.axis("on")
plt.suptitle("Resized images")

#### d) Augmentera datan. Varför behövs dataaugmentering och hur beslutar vilka parametrar du valt för augmenteringen?


- When the amount of data is insufficient, the model is easy to overfit, and the accuracy can not improve. Therefore, data augmentation is needed. By using augmentation, it can automatically enhance training data.

- Here I decided to adjust the following three parameters:

(1) Color.  t.ex. "featurewise_center"

(2) Geometry.  t.ex. "shear_range"

(3) Bounding box.  t.ex. "width_shift_range"

In [25]:
scaled_X_train_small_resized = X_train_small_resized.astype("float32") / 255
scaled_X_val_small_resized = X_val_small_resized.astype("float32") / 255
scaled_X_test_small_resized = X_test_small_resized.astype("float32") / 255

scaled_X_train_tiny_resized = X_train_tiny_resized.astype("float32") / 255
scaled_X_val_tiny_resized = X_val_tiny_resized.astype("float32") / 255
scaled_X_test_tiny_resized = X_test_tiny_resized.astype("float32") / 255

In [26]:
train_image_generator  = ImageDataGenerator(
    rotation_range = 30,      # random rotation of the image. 40 degree
    width_shift_range = 0.2,  # is the extent to which the image is panned horizontally or vertically
    height_shift_range= 0.2,
    shear_range= 0.2,         # the angle of the random staggered transformation.
    zoom_range= 0.1,          # zoom in/zoom out
    horizontal_flip= True,    # reverse
    featurewise_center=True   # Color change
)

In [27]:
scaled_X_test_small_resized.shape

In [28]:
# We generate only train data for small and tiny

train_small_generator = train_image_generator.flow(scaled_X_train_small_resized, y_train_small, batch_size=32)
test_small_generator = ImageDataGenerator()
val_small_generator = test_small_generator.flow(scaled_X_val_small_resized, y_val_small, batch_size=32)

train_tiny_generator = train_image_generator.flow(scaled_X_train_tiny_resized, y_train_tiny, batch_size=32)
test_tiny_generator = ImageDataGenerator()
val_tiny_generator = test_tiny_generator.flow(scaled_X_val_tiny_resized, y_val_tiny, batch_size=32)

For small data:

In [29]:
print(len(train_small_generator.next()))

sample_batch = train_small_generator.next()
print(sample_batch[0].shape) # 32 samples in a batch

plt.imshow(sample_batch[0][7])
plt.suptitle("Sample of augmentation")

# 2. Träna modeller (*)


#### För uppgifterna nedan, jobba med dataseten en åt gången:

Small

- utan augmenterig 
- med augmentering

Tiny (BONUS)

- utan augmentering
- med augmentering

#### a) Använd följande nätverk och träna på datan. Gör hyperparametertuning för några parametrar (beskriv hur du gör). Visualisera och analysera loss-kurvor, accuracy-kurvor.

Create CNN model with layers:

In [30]:
def CNN_model(drop_rate, learning_rate):

  model = models.Sequential() 

  adam = Adam(learning_rate=learning_rate)
  
  # Biuld Conv2D and MaxPooling2D layers:
  # The depth of the feature map is increasing (from 32 to 128)
  model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3))) # same shape as we created
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(64, (3, 3), activation='relu')) 
  model.add(MaxPooling2D((2, 2))) 
  model.add(Conv2D(128, (3, 3), activation='relu')) 
  model.add(MaxPooling2D((2, 2))) 
  model.add(Conv2D(128, (3, 3), activation='relu')) 
  model.add(MaxPooling2D((2, 2)))

  # connect to Flatten layer
  model.add(Flatten())
  model.add(Dropout(drop_rate)) # Dropout layer can reduce overfitting
  model.add(Dense(512, activation='relu')) 
  model.add(Dense(1, activation='sigmoid'))

  model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['acc'])
  # using binary crossentropy here because of sigmoid function (binary labels)

  return model


model = CNN_model(0.3, 0.001)
model.summary()

Create a function for plot:

In [31]:
def plot_metrics(metrics):
    _, ax = plt.subplots(1,2, figsize = (12,4))
    metrics[["loss", "val_loss"]].plot(ax = ax[0], grid = True)
    metrics[["acc", "val_acc"]].plot(ax = ax[1], grid = True)

#### For tiny dataset without augmentation:

In [32]:
steps_per_epoch_tiny = int(len(scaled_X_train_tiny_resized)/32)
validation_steps_tiny = int(len(scaled_X_val_tiny_resized)/32)

steps_per_epoch_tiny, validation_steps_tiny

In [33]:
# use ImageDataGenerator() for a non-augmentation dataset

test_tiny_generator_0 = ImageDataGenerator()
train_tiny_generator_0 = test_tiny_generator_0.flow(scaled_X_train_tiny_resized, y_train_tiny, batch_size=32)
val_tiny_generator_0 = test_tiny_generator_0.flow(scaled_X_val_tiny_resized, y_val_tiny, batch_size=32)

In [34]:
# Here choose "patience = 10, restore_best_weights = True, epochs = 50"
early_stopper = EarlyStopping(monitor = "val_acc", mode = "max", patience = 10, restore_best_weights = True)

model.fit(
    train_tiny_generator_0,
    steps_per_epoch = steps_per_epoch_tiny,
    epochs = 50,
    callbacks = [early_stopper],
    validation_data = val_tiny_generator_0,
    validation_steps = validation_steps_tiny
)

In [35]:
metrics_tiny_without_augmentation = pd.DataFrame(model.history.history)
plot_metrics(metrics_tiny_without_augmentation)
plt.suptitle("Train/validation loss & Train/validation acc")

#### For tiny data with augmentation:

In [36]:
# Here choose "patience = 10, restore_best_weights = True, epochs = 50"
early_stopper = EarlyStopping(monitor = "val_acc", mode = "max", patience = 10, restore_best_weights = True)

model.fit(
    train_tiny_generator,
    steps_per_epoch = steps_per_epoch_tiny,
    epochs = 50,
    callbacks = [early_stopper],
    validation_data = val_tiny_generator,
    validation_steps = validation_steps_tiny
)

In [37]:
metrics_tiny = pd.DataFrame(model.history.history)
plot_metrics(metrics_tiny)
plt.suptitle("Train/validation loss & Train/validation acc")

#### For small dataset without augmentation

In [38]:
steps_per_epoch_small = int(len(scaled_X_train_small_resized)/32)
validation_steps_small = int(len(scaled_X_val_small_resized)/32)

steps_per_epoch_small, validation_steps_small

In [39]:
test_small_generator_0 = ImageDataGenerator()
train_small_generator_0 = test_small_generator_0.flow(scaled_X_train_small_resized, y_train_small, batch_size=32)
val_small_generator_0 = test_small_generator_0.flow(scaled_X_val_small_resized, y_val_small, batch_size=32)

In [40]:
# patience = 10, restore_best_weights = True

early_stopper = EarlyStopping(monitor = "val_acc", mode = "max", patience = 10, restore_best_weights = True)

model.fit(
    train_small_generator_0,
    steps_per_epoch = steps_per_epoch_small,
    epochs = 50,
    callbacks = [early_stopper],
    validation_data = val_small_generator_0,
    validation_steps = validation_steps_small
)

In [41]:
metrics_small_without_augmentation = pd.DataFrame(model.history.history)
plot_metrics(metrics_small_without_augmentation)
plt.suptitle("Train/validation loss & Train/validation acc")

The model fit well with train data but not with validation data.

#### For small dataset with augmentation

In [42]:
early_stopper = EarlyStopping(monitor = "val_acc", mode = "max", patience = 10, restore_best_weights = True)

model.fit(
    train_small_generator,
    steps_per_epoch = steps_per_epoch_small,
    epochs = 50,
    callbacks = [early_stopper],
    validation_data = val_small_generator,
    validation_steps = validation_steps_small
)

In [43]:
metrics_small = pd.DataFrame(model.history.history)
plot_metrics(metrics_small)
plt.suptitle("Train/validation loss & Train/validation acc")

#### b) Förändra nätverket i a) experimentera och ändra i lite komponenter. Beskriv vad du ändrar och varför. Glöm inte att evaluera på valideringsdatan.


Model_2 try to use 3 Conv2D & MaxPooling 2D (64,128,128), drop_rate = 0.4, learning_rate = 0.002

The number of convolution kernels change to 64 on the first step. The reason for this is to set more information from the start.

Try to increase the features in the beginning.

In [44]:
def CNN_model_2(drop_rate, learning_rate):

  model = models.Sequential() 

  adam = Adam(learning_rate=learning_rate)
  
  # Biuld Conv2D and MaxPooling2D layers:
  # The depth of the feature map is increasing (64, 128, 128)

  model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(256, 256, 3)))
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(128, (3, 3), activation='relu')) 
  model.add(MaxPooling2D((2, 2))) 
  model.add(Conv2D(128, (3, 3), activation='relu')) 
  model.add(MaxPooling2D((2, 2))) 

  model.add(Flatten())
  model.add(Dropout(drop_rate))
  model.add(Dense(512,activation='relu'))
  model.add(Dense(1,activation='sigmoid'))
  
  model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['acc'])
  # using binary crossentropy here because of sigmoid function (binary labels)

  return model


model_2 = CNN_model_2(0.4, 0.002)
model_2.summary()

***Model_2 for tiny data without augmentation:***

In [45]:
early_stopper = EarlyStopping(monitor = "val_acc", mode = "max", patience = 10, restore_best_weights = True)

model_2.fit(
    train_tiny_generator_0,
    steps_per_epoch = steps_per_epoch_tiny,
    epochs = 50,
    callbacks = [early_stopper],
    validation_data = val_tiny_generator_0,
    validation_steps = validation_steps_tiny
)

In [46]:
metrics_tiny_2 = pd.DataFrame(model_2.history.history)
plot_metrics(metrics_tiny_2)
plt.suptitle("Train/validation loss & Train/validation acc for tiny model_2")

***Model_2 for tiny data with augmentation:***

In [47]:
early_stopper = EarlyStopping(monitor = "val_acc", mode = "max", patience = 10, restore_best_weights = True)

model_2.fit(
    train_tiny_generator,
    steps_per_epoch = steps_per_epoch_tiny,
    epochs = 50,
    callbacks = [early_stopper],
    validation_data = val_tiny_generator,
    validation_steps = validation_steps_tiny
)

In [48]:
metrics_tiny_2 = pd.DataFrame(model_2.history.history)
plot_metrics(metrics_tiny_2)
plt.suptitle("Train/validation loss & Train/validation acc for tiny_aug model_2")

***Model_2 for Small data without augmentation:***

In [49]:
early_stopper = EarlyStopping(monitor = "val_acc", mode = "max", patience = 10, restore_best_weights = True)

model_2.fit(
    train_small_generator_0,
    steps_per_epoch = steps_per_epoch_small,
    epochs = 50,
    callbacks = [early_stopper],
    validation_data = val_small_generator_0,
    validation_steps = validation_steps_small
)

In [50]:
metrics_small_2 = pd.DataFrame(model_2.history.history)
plot_metrics(metrics_small_2)
plt.suptitle("Train/validation loss & Train/validation acc for small model_2")

***Model_2 for Small data with augmentation:***

In [51]:
early_stopper = EarlyStopping(monitor = "val_acc", mode = "max", patience = 10, restore_best_weights = True)

model_2.fit(
    train_small_generator,
    steps_per_epoch = steps_per_epoch_small,
    epochs = 50,
    callbacks = [early_stopper],
    validation_data = val_small_generator,
    validation_steps = validation_steps_small
)

In [52]:
metrics_small_2 = pd.DataFrame(model_2.history.history)
plot_metrics(metrics_small_2)
plt.suptitle("Train/validation loss & Train/validation acc for small_aug model_2")

Model_3 try to use 4 Conv2D & MaxPooling 2D (64,128,128,256), drop_rate = 0.5, learning_rate = 0.001

The number of convolution kernels change to 32 in the first step and add a 256 kernel. I try to add more information at the end of filtering.

In [53]:
def CNN_model_3(drop_rate, learning_rate):

  model = models.Sequential() 

  adam = Adam(learning_rate=learning_rate)
  
  # Biuld Conv2D and MaxPooling2D layers:
  # The depth of the feature map is increasing (32, 64, 128, 128, 256)

  model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)))
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(64, (3, 3), activation='relu'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(128, (3, 3), activation='relu')) 
  model.add(MaxPooling2D((2, 2))) 
  model.add(Conv2D(128, (3, 3), activation='relu')) 
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(256, (3, 3), activation='relu')) 
  model.add(MaxPooling2D((2, 2)))

  model.add(Flatten())
  model.add(Dropout(drop_rate))
  model.add(Dense(512,activation='relu'))
  model.add(Dense(1,activation='sigmoid'))
  
  model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['acc'])
  # using binary crossentropy here because of sigmoid function (binary labels)

  return model


model_3 = CNN_model_3(0.5, 0.001)
model_3.summary()

***Model_3 for tiny data without augmentation:***

In [54]:
early_stopper = EarlyStopping(monitor = "val_acc", mode = "max", patience = 10, restore_best_weights = True)

model_3.fit(
    train_tiny_generator_0,
    steps_per_epoch = steps_per_epoch_tiny,
    epochs = 50,
    callbacks = [early_stopper],
    validation_data = val_tiny_generator_0,
    validation_steps = validation_steps_tiny
)

In [55]:
metrics_tiny_3 = pd.DataFrame(model_3.history.history)
plot_metrics(metrics_tiny_3)
plt.suptitle("Train/validation loss & Train/validation acc for tiny model_3")

***Model_3 for tiny data with augmentation:***

In [56]:
early_stopper = EarlyStopping(monitor = "val_acc", mode = "max", patience = 10, restore_best_weights = True)

model_3.fit(
    train_tiny_generator,
    steps_per_epoch = steps_per_epoch_tiny,
    epochs = 50,
    callbacks = [early_stopper],
    validation_data = val_tiny_generator,
    validation_steps = validation_steps_tiny
)

In [57]:
metrics_tiny_3 = pd.DataFrame(model_3.history.history)
plot_metrics(metrics_tiny_3)
plt.suptitle("Train/validation loss & Train/validation acc for tiny_aug model_3")

***Model_3 for small data without augmentation:***

In [58]:
early_stopper = EarlyStopping(monitor = "val_acc", mode = "max", patience = 10, restore_best_weights = True)

model_3.fit(
    train_small_generator_0,
    steps_per_epoch = steps_per_epoch_small,
    epochs = 50,
    callbacks = [early_stopper],
    validation_data = val_small_generator_0,
    validation_steps = validation_steps_small
)

In [59]:
metrics_small_3 = pd.DataFrame(model_3.history.history)
plot_metrics(metrics_small_3)
plt.suptitle("Train/validation loss & Train/validation acc for small model_3")

***Model_3 for small data with augmentation:***

In [60]:
early_stopper = EarlyStopping(monitor = "val_acc", mode = "max", patience = 10, restore_best_weights = True)

model_3.fit(
    train_small_generator,
    steps_per_epoch = steps_per_epoch_small,
    epochs = 50,
    callbacks = [early_stopper],
    validation_data = val_small_generator,
    validation_steps = validation_steps_small
)

In [61]:
metrics_small_3 = pd.DataFrame(model_3.history.history)
plot_metrics(metrics_small_3)
plt.suptitle("Train/validation loss & Train/validation acc for small_aug model_3")

**Summary**:

- Model_1: One layer removed(32 features) does not have much effect on the model. The drop rate is increased to 0.4 and the learning rate is increased to 0.002
- Model_2: When adding a layer (256 features), the model is greatly improved. The increase in accuracy is significant. The drop rate is increased to 0.5 and the learning rate is restored to 0.001
- The step and the input_shape were not changed. Only the effect of layers on models is compaired here.
- Note that the small data perform better than tiny data on both models with Aug/non-aug, because of the data size is limit.
- The accuracy or loss of training on original data can be higher than the accuracy or loss on augmentation data, but on the validation data, augmentayion perform better than original data on both models. The stability and fitness of augmentation data are better.

#### c) Välj en modell, träna på tränings- och valideringsdatan. Gör inferens på testdatan och utvärdera din modell.


Model_3 will be choosen

In [62]:
# Concatenate the training and validation data

X_train_val_small = np.concatenate((scaled_X_train_small_resized, scaled_X_val_small_resized))
y_train_val_small = np.concatenate((y_train_small, y_val_small))
X_train_val_small.shape, y_train_val_small.shape

In [63]:
train_val_small_generator = train_image_generator.flow(X_train_val_small, y_train_val_small, batch_size=32)

In [64]:
early_stopper = EarlyStopping(monitor = "val_acc", mode = "max", patience = 10, restore_best_weights = True)

model_3.fit(
    train_val_small_generator,
    steps_per_epoch = steps_per_epoch_small,
    epochs = 50,
    callbacks = [early_stopper]
)

#### *Valuation*

In [65]:
y_pred_small_3 = model_3.predict(scaled_X_test_small_resized)
y_pred_small_3 = ((y_pred_small_3 > 0.5)+0).ravel() # transfer probabilities to 0/1
y_test_small = np.array(y_test_small)

print(classification_report(y_test_small, y_pred_small_3))
cm = confusion_matrix(y_test_small, y_pred_small_3)
ConfusionMatrixDisplay(cm).plot()

*Accuracy reach to about 90%, extremely good!*

Prediction test with my image

In [97]:
huihui = plt.imread("../input/huihui/huihui.jpg")
plt.imshow(huihui)

In [98]:
huihui = cv2.resize(huihui, (256, 256))
huihui_scaled = huihui.astype("float32") / 255
huihui_scaled = huihui_scaled[None,:,:,:] # add a dimension
huihui_scaled.shape

In [100]:
huihui_p = model_3.predict(huihui_scaled)
huihui_p = ((huihui_p > 0.5)+0).ravel()
huihui_p

#### d) Tag ett eller flera av nätverken (VGG16, Resnet, Xception, Inception), läs deras forskningsartikel, sammanfatta kort ca 1/2 - 1 sida för en artikel. Utför därefter transfer learning och evaluera din modell. Beskriv också hur transfer learning fungerar.


##### VGG16 summary:

- VGG16 is a model proposed by the Visual Geometry Group of Oxford University.
- VGG16 is characterized by proving that small convolutions can effectively improve performance by increasing the network depth.
- The default size of VGG16 for input images is 224 * 224 * 3. The VGG16 network structure has a total of 16 layers (with parameters), that is, 13 convolutional layers, 5 pooling layers(without parameters), and 3 fully connected layers, excluding the activation function(layers)
- conv3-64 means that the dimension becomes 64 after the third layer of convolution. Similarly, conv3-128 means that the dimension becomes 128 after the third layer of convolution
- FC-4096 refers to 4096 nodes in the fully connected layer, FC-1000 is a fully connected layer with 1000 nodes

|             Modules         | The layers of each module |
|-----------------------------|---------------------------|
|                Input images |             224 * 224 * 3     |
| | |
|||
|                 Module_1    |               **conv3-64**|
|                             |               **conv3-64**    |
|                             |               maxpool     |
|                 Module_2    |               **conv3-128**   |
|                             |               **conv3-128**   |
|                             |               maxpool     |
|                 Module_3    |               **conv3-256**   |
|                             |               **conv3-256**   |
|                             |               **conv3-256**   |
|                             |               maxpool     |
|                 Module_4    |               **conv3-512**   |
|                             |               **conv3-512**   |
|                             |               **conv3-512**   |
|                             |               maxpool     |
|                 Module_5    |               **conv3-512**   |
|                             |               **conv3-512**   |
|                             |               **conv3-512**   |
|                             |               maxpool     |
|      Module_6 (full connection layer and output layer)           |          (Flatten)         |
|                |          **FC-4096**          |
|                |          **FC-4096**          |
|                |          **FC-1000**  (classification)        |
|                |          softmax (Output function)          |

Process: 224 * 224 * 3 input: 
- conv1_1+conv1_2+pool_1：1st conv1, (3 * 3 * 3) * 64 = 1728(training parameters) --- 2st conv1（3 * 3 * 64） * 64=36864(training parameters); after maxpool(2 * 2), size: 112 * 112 * 64
- conv2_1+conv2_2+pool_2：after 2st conv2,（3 * 3 * 128） * 128=147456(training parameters); after maxpool, size: 56 * 56 * 128
- conv3_1+conv3_2+conv3_3+pool_3：after 3st conv3,（3 * 3 * 256） * 256=589824(training parameters); after maxpool, size: 28 * 28 * 256
- conv4_1+conv4_2+conv4_3+pool_4：after 3st conv4,（3 * 3 * 512） * 512=2359296(training parameters); after maxpool, size: 14 * 14 * 512
- conv5_1+conv5_2+conv5_3+pool_5：after 3st conv5,（3 * 3 * 512） * 512=2359296(training parameters); after maxpool, size: 7 * 7 * 512

In [101]:
from tensorflow.keras.applications import VGG16

input_shape = 256, 256, 3

base_model = VGG16(weights="imagenet", include_top=False,
                   input_shape=(input_shape))

base_model.summary()

In [102]:
model_transfer = Sequential([
    base_model,
    Flatten(),
    Dropout(.5),
    Dense(256, activation="relu", kernel_initializer="he_normal"),
    Dropout(.5),
    Dense(1, activation="sigmoid")
], name = "Transfer_model")

for layer in base_model.layers:
    layer.trainable = False

model_transfer.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["acc"])

model_transfer.summary()

Here applying relevant parts of a pre-trained machine learning model(VGG16) to our problem. Which "base_model" is the core information (operation) of the model, and new layers(flatten,dropout,denses) are added to the model to our cats/dogs tasks.

In [103]:
model_transfer.fit(train_tiny_generator_0, epochs = 20, validation_data = val_tiny_generator_0)

In [106]:
metrics = pd.DataFrame(model_transfer.history.history)
metrics.tail()

In [105]:
metrics[["acc", "val_acc"]].plot()

References: 
- https://www.kaggle.com/code/rkb0023/cnn-classifier-cats-and-dogs
- https://www.kaggle.com/code/utshabkumarghosh/dogvcat2/notebook
- https://www.kaggle.com/code/akshat4112/cats-vs-dogs-using-convnets-transfer-learning
- https://arxiv.org/pdf/1409.1556.pdf
- https://blog.csdn.net/hgnuxc_1993/article/details/115956774?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522165238847816781483760778%2522%252C%2522scm%2522%253A%252220140713.130102334..%2522%257D&request_id=165238847816781483760778&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~top_positive~default-1-115956774-null-null.142^v9^pc_search_result_control_group,157^v4^control&utm_term=vgg16%E7%BD%91%E7%BB%9C%E7%BB%93%E6%9E%84%E8%AF%A6%E8%A7%A3&spm=1018.2226.3001.4187